In [1]:
import gensim
import pickle
import tensorflow as tf
import numpy as np
from create_batch import mini_batch
import os
import sys
import re
import enjoy_tree
tf.set_random_seed(10)

In [2]:
with open('Sample_data.bin', 'rb') as f:
    batches_morphene = pickle.load(f)
    batches_arc = pickle.load(f)
    batches_relation = pickle.load(f)
    dictionary_tag = pickle.load(f)
    dictionary_rel = pickle.load(f)
    dictionary_char = pickle.load(f)
    label_dim = pickle.load(f)
with open('gensim_embedding_100.txt', 'rb') as f:
    model = pickle.load(f)
def get_vector(word):
    vector = model.wv[word]
    return vector

In [3]:
tag_dim = 100
char_dim = len(dictionary_char)
word_max_length = 32
word_vec_dim = 100
char_vec_dim = 100
concat_dim = 200
input_dim = 100
sequence_length = 220
in_size = concat_dim
hidden_size = 200
arc_size = 400
rel_size = 100
dense_size = 64
regularization_scale = 0.001
beta = 0.009
epsilon = 2e-3
grad_clip = 5
dropout_rate = 0.33
learning_rate = 0.002
batch_size = 1
unroll_step = sequence_length
regularizer = tf.contrib.layers.l2_regularizer(scale=regularization_scale)

In [4]:
train_inputs_vec, train_inputs_tag, train_inputs_char, train_labels_arc, train_labels_rel = [], [], [], [], []
training = tf.placeholder(dtype=tf.bool, name="training")
for ui in range(unroll_step):
    train_inputs_vec.append(tf.placeholder(tf.float32, shape=[None, input_dim], name='train_inputs_vec_%d'%ui))
    train_inputs_tag.append(tf.placeholder(tf.float32, shape=[None, input_dim], name='train_inputs_tag_%d'%ui))
    train_inputs_char.append(tf.placeholder(tf.float32, shape=[None, word_max_length, char_dim, 1], name='train_inputs_char_%d'%ui))
prev_train_h_fd = tf.Variable(tf.zeros([batch_size, hidden_size],dtype=tf.float32),name='train_h_fd',trainable=False)
prev_train_h_bd = tf.Variable(tf.zeros([batch_size, hidden_size],dtype=tf.float32),name='train_h_bd',trainable=False)
train_labels_arc = tf.placeholder(tf.float32, shape=[batch_size, sequence_length, sequence_length], name='train_labels_arc')
train_labels_rel = tf.placeholder(tf.float32, shape=[batch_size, sequence_length, label_dim], name='train_labels_rel')

In [5]:
with tf.variable_scope('a'):
    w_forward = tf.get_variable(name="w_forward", shape=[3,in_size, hidden_size], initializer=tf.contrib.layers.xavier_initializer(), regularizer=regularizer)
    u_forward = tf.get_variable(name="u_forward", shape=[3,hidden_size, hidden_size], initializer=tf.contrib.layers.xavier_initializer(), regularizer=regularizer)
    b_forward = tf.get_variable(name="b_forward", shape=[3,hidden_size], initializer=tf.zeros_initializer())
    w_backward = tf.get_variable(name="w_backward", shape=[3,in_size, hidden_size], initializer=tf.contrib.layers.xavier_initializer(), regularizer=regularizer)
    u_backward = tf.get_variable(name="u_backward", shape=[3,hidden_size, hidden_size], initializer=tf.contrib.layers.xavier_initializer(), regularizer=regularizer)
    b_backward = tf.get_variable(name="b_backward", shape=[3,hidden_size], initializer=tf.zeros_initializer())

In [6]:
with tf.variable_scope('b'):
    w_arc_dep = tf.get_variable(name="w_arc_dep", shape=[hidden_size, arc_size], initializer=tf.contrib.layers.xavier_initializer(), regularizer=regularizer)
    b_arc_dep = tf.get_variable(name="b_arc_dep", shape=[arc_size], initializer=tf.zeros_initializer())
    w_arc_head = tf.get_variable(name="w_arc_head", shape=[hidden_size, arc_size], initializer=tf.contrib.layers.xavier_initializer(), regularizer=regularizer)
    b_arc_head = tf.get_variable(name="b_arc_head", shape=[arc_size], initializer=tf.zeros_initializer())
    w_rel_dep = tf.get_variable(name="w_rel_dep", shape=[hidden_size, rel_size], initializer=tf.contrib.layers.xavier_initializer(), regularizer=regularizer)
    b_rel_dep = tf.get_variable(name="b_rel_dep", shape=[rel_size], initializer=tf.zeros_initializer())
    w_rel_head = tf.get_variable(name="w_rel_head", shape=[hidden_size, rel_size], initializer=tf.contrib.layers.xavier_initializer(), regularizer=regularizer)
    b_rel_head = tf.get_variable(name="b_rel_head", shape=[rel_size], initializer=tf.zeros_initializer())

In [7]:
with tf.variable_scope('biaffine_classifier'):
    w_arc = tf.get_variable(name="w_arc", shape=[arc_size, arc_size], initializer=tf.contrib.layers.xavier_initializer(), regularizer=regularizer)
    b_arc = tf.get_variable(name="b_arc", shape=[arc_size, 1], regularizer=regularizer)
    w_rel = tf.get_variable(name="w_rel", shape=[label_dim, concat_dim], initializer=tf.contrib.layers.xavier_initializer(), regularizer=regularizer)
    u_rel = tf.get_variable(name="u_rel", shape=[rel_size, rel_size], initializer=tf.contrib.layers.xavier_initializer(), regularizer=regularizer)
    b_rel = tf.get_variable(name="b_rel", shape=[label_dim, 1], regularizer=regularizer)

In [8]:
CHOSUNG_LIST = ['ᄀ', 'ᄁ', 'ᄂ', 'ᄃ', 'ᄄ', 'ᄅ', 'ᄆ', 'ᄇ', 'ᄈ', 'ᄉ', 'ᄊ', 'ᄋ', 'ᄌ', 'ᄍ', 'ᄎ', 'ᄏ', 'ᄐ', 'ᄑ', 'ᄒ']

JUNGSUNG_LIST = ['ᅡ', 'ᅢ', 'ᅣ', 'ᅤ', 'ᅥ', 'ᅦ', 'ᅧ', 'ᅨ', 'ᅩ', 'ᅪ', 'ᅫ', 'ᅬ', 'ᅭ', 'ᅮ', 'ᅯ', 'ᅰ', 'ᅱ', 'ᅲ', 'ᅳ', 'ᅴ','ᅵ']

JONGSUNG_LIST = [' ', 'ᆨ', 'ᆩ', 'ᆪ', 'ᆫ', 'ᆬ', 'ᆭ', 'ᆮ', 'ᆯ', 'ᆰ', 'ᆱ', 'ᆲ', 'ᆳ', 'ᆴ', 'ᆵ', 'ᆶ', 'ᆷ', 'ᆸ', 'ᆹ', 'ᆺ','ᆻ', 'ᆼ', 'ᆽ', 'ᆾ', 'ᆿ', 'ᇀ', 'ᇁ', 'ᇂ']

INDI_LIST = ['ㄱ', 'ㄲ', 'ㄳ', 'ㄴ', 'ㄵ', 'ㄶ', 'ㄷ', 'ㄸ', 'ㄹ', 'ㄺ', 'ㄻ', 'ㄼ', 'ㄽ', 'ㄾ', 'ㄿ', 'ㅀ', 'ㅁ', 'ㅂ', 'ㅃ', 'ㅄ', 'ㅅ',
             'ㅆ', 'ㅇ', 'ㅈ', 'ㅉ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ', 'ㅏ', 'ㅐ', 'ㅑ', 'ㅒ', 'ㅓ', 'ㅔ', 'ㅕ', 'ㅖ', 'ㅗ', 'ㅘ', 'ㅙ', 'ㅚ',
             'ㅛ', 'ㅜ', 'ㅝ', 'ㅞ', 'ㅟ', 'ㅠ', 'ㅡ', 'ㅢ', 'ㅣ']

hangul_johab = range(44032,55204)
hangul_jaeum = range(12593,12623)
hangul_moeum = range(12623,12644)
hangul_chosung = range(4352,4371)
hangul_jungsung = range(4449,4470)
hangul_jongsung = range(4520,4547)
english1 = range(65,91)
english2 = range(97,123)
digit = range(48,58)
special_char = [ord('.'), ord('\''), ord('?'), ord(','), ord('!'), ord('%')]

def syllable(char):
    s = ord(char) - 44032
    cho = (s//21)//28
    jung = (s%(21*28))//28
    jong = (s%28)
    
    return CHOSUNG_LIST[cho], JUNGSUNG_LIST[jung], JONGSUNG_LIST[jong]

def split_data(batches):
    batches_word_vector = []
    batches_character_jaso = []
    batches_tag = []
    batches_vector_append = batches_word_vector.append
    batches_jaso_append = batches_character_jaso.append
    batches_tag_append = batches_tag.append
    UNK_TOKEN = np.zeros(word_vec_dim, np.float32)
    PAD_VECTOR = np.zeros(word_vec_dim, np.float32)
    JASO_PAD = np.zeros(word_max_length, np.float32)
    for line in batches:
        batch_jaso = []
        batch_wordvector = []
        batch_tag = []
        t_append = batch_tag.append
        v_append = batch_wordvector.append

        for w_ in line:
            v = re.sub(r"/{1}([A-Z]+)", r" \1", w_)
            v = v.split(' ')
            if v[0] in model.wv.vocab:
                v_append(get_vector(v[0]))
            else:
                v_append(UNK_TOKEN)
            t_append(v[-1])
            word = []
            w_append = word.append
            w_extend = word.extend
            for c in v[0]:
                    sign_unk = 0
                    if ord(c) in hangul_johab or ord(c) in hangul_chosung or \
                       ord(c) in hangul_jungsung or ord(c) in hangul_jongsung or \
                       ord(c) in hangul_jaeum or ord(c) in hangul_moeum or \
                       ord(c) in english1 or ord(c) in english2 or \
                       ord(c) in digit or ord(c) in special_char: pass
                    else: sign_unk = 1

                    if sign_unk == 1:
                        w_append('<UNK>')
                    else:
                        if ord(c) in hangul_johab:
                            jaso = syllable(c)
                            if ' ' in jaso:
                                jaso = jaso[0:-1]
                            w_extend(jaso)
                        else:
                            w_append(c)
            batch_jaso.append(word)
        batches_jaso_append(batch_jaso)
        batches_tag_append(batch_tag)
        batches_vector_append(batch_wordvector)

    for i in range(len(batches_word_vector)):
        present_length = len(batches_word_vector[i])
        for _ in range(present_length, sequence_length):
            batches_word_vector[i].append(PAD_VECTOR)

    indexed_batches = []
    batches_jaso = []
    d_append = indexed_batches.append
    for sentence in batches_character_jaso:
        sen = []
        s_append = sen.append
        for word in sentence:
            s_append(([dictionary_char[char] for char in word]))
        d_append(sen)

    for i in range(len(indexed_batches)):
        sequence = []
        sentence_length = len(indexed_batches[i])
        for j in range(sentence_length):
            present_length = len(indexed_batches[i][j])
            padding_length = word_max_length - present_length
            if(padding_length % 2 == 0):
                left = right = padding_length // 2
            else:
                right = padding_length // 2
                left = right + 1
            sequence.append(np.pad(indexed_batches[i][j], (left, right), 'constant', constant_values=0))
        for k in range(sentence_length, sequence_length):
            sequence.append(JASO_PAD)
        batches_jaso.append(sequence)

    batches_jaso_input = np.zeros([len(batches), sequence_length, word_max_length, char_dim])
    for batch_index in range(len(batches_jaso)):
        for sequence_index in range(len(batches_jaso[batch_index])):
            for word_index in range(len(batches_jaso[batch_index][sequence_index])):
                char_index = batches_jaso[batch_index][sequence_index][word_index]
                if char_index != 0.0:
                    batches_jaso_input[batch_index][sequence_index][word_index][char_index] = 1
    batches_jaso_input = np.reshape(batches_jaso_input, [len(batches), sequence_length, word_max_length, char_dim, 1])

    batches_tag_onehot = []
    for _, batch in enumerate(batches_tag):
        indices = [0 for i in range(len(batch))]
        for idx, tag in enumerate(batch):
            if tag in dictionary_tag:
                indices[idx] = dictionary_tag[tag]
            else:
                indices[idx] = 0
        present_length = len(indices)
        batch_tag_onehot = []
        for i in range(present_length):
            p = indices[i] 
            onehot = np.zeros(100, np.float32)
            onehot[p] = 1.0
            batch_tag_onehot.append(onehot)
        batches_tag_onehot.append(batch_tag_onehot)

    for i in range(len(batches_tag_onehot)):
        present_length = len(batches_tag_onehot[i])
        for _ in range(present_length, sequence_length):
            batches_tag_onehot[i].append(PAD_VECTOR)

    return np.transpose(np.asarray(batches_word_vector), [1,0,2]), np.transpose(batches_jaso_input, [1,0,2,3,4]), np.transpose(np.asarray(batches_tag_onehot), [1,0,2])

In [9]:
def layer_norm(inputs,scope,scale=True,layer_norm=True,epsilon = 1e-5):
    
    if layer_norm == True:
        
        with tf.variable_scope(scope, reuse=tf.AUTO_REUSE):
            
            if scale == False:
                scale = tf.ones([inputs.get_shape()[1]],tf.float32)
            else:
                scale = tf.get_variable("scale", shape=[inputs.get_shape()[1]],
                        initializer=tf.ones_initializer())
        
        

        mean, var = tf.nn.moments(inputs, [1], keep_dims=True)
        
        LN = tf.multiply((scale / tf.sqrt(var + epsilon)),(inputs - mean))
        
        return LN
    else:
        return inputs

In [10]:
def get_score_rel(H_rel_head, H_rel_dep, prediction_head, w, u, b, seq_len, scope):
    
    batch_size = tf.shape(H_rel_head)[0]
    b_hidden_lists = tf.TensorArray(size=batch_size,dtype=tf.float32)
    j=0
    def cond(i,hidden_lists):
        return i < seq_len
    
    def body(i,hidden_lists):
        h_rel_head = H_rel_head[j][prediction_head[j][i]]
        h_rel_dep = H_rel_dep[j][i]
        h_rel_head = tf.reshape(h_rel_head, [rel_size, 1])
        h_rel_dep = tf.reshape(h_rel_dep, [rel_size, 1])
        s = tf.matmul(tf.matmul(tf.transpose(h_rel_head, [1, 0]), u), h_rel_dep) + tf.matmul(w, tf.concat([h_rel_dep, h_rel_head], 0)) + b

        hidden_lists = hidden_lists.write(i,s)
        return i+1,hidden_lists
    
    def b_cond(j, b_hidden_lists):
        return j < batch_size
    
    def b_body(j, b_hidden_lists):
        hidden_lists = tf.TensorArray(size=seq_len,dtype=tf.float32)
        i = 0
        _,hidden_lists = tf.while_loop(cond, body, [i, hidden_lists], swap_memory=True, name="get_score_rel_inner")
        hidden_lists = hidden_lists.stack()
        b_hidden_lists = b_hidden_lists.write(j,hidden_lists)
        return j+1, b_hidden_lists
    
    _, b_hidden_lists = tf.while_loop(b_cond, b_body, [j,b_hidden_lists], swap_memory=True, name="get_score_rel_outter")
    return b_hidden_lists.stack()

In [11]:
def conv2d_batch(x, word_max_length, char_dim, seq_len, scope):
    filter_size = [3,5,7,9]
    num_filters = 25
    conved_layers = []
    for f in filter_size:
        Y = tf.layers.conv2d(x, filters=num_filters, kernel_size=[f, char_dim], activation=tf.nn.relu, padding= 'VALID',
                                  kernel_initializer=tf.truncated_normal_initializer(stddev=0.01))
        Y = tf.layers.max_pooling2d(Y, pool_size=[word_max_length - f + 1, 1], strides=2, padding='VALID')
        conved_layers.append(Y)
    concat_chars = tf.concat(conved_layers,3)
    character_embd = tf.reshape(concat_chars,[-1, char_vec_dim])
    return character_embd

In [12]:
def gru_cell(x, hidden, scope):
    z = tf.sigmoid(layer_norm( tf.matmul(x,w_forward[0]) + tf.matmul(hidden,u_forward[0]) , scope+"_z")+ b_forward[0])
    r = tf.sigmoid(layer_norm( tf.matmul(x,w_forward[1]) + tf.matmul(hidden,u_forward[1]) , scope+"_r")+ b_forward[1])
    h_ = tf.tanh(layer_norm( tf.matmul(x,w_forward[2]) + tf.multiply(r,tf.matmul(hidden,u_forward[2])),scope+"_h") + b_forward[2])
    hidden = tf.multiply(z,h_) + tf.multiply((1-z),hidden) 
    return hidden

In [13]:
with tf.name_scope("layer_1"):
    train_inputs = list()
    for X_VEC, X_TAG, X_CHAR in zip(train_inputs_vec, train_inputs_tag, train_inputs_char):
        cnn_embd = conv2d_batch(X_CHAR, word_max_length, char_dim, sequence_length, "conv2d")
        word_embd = tf.add(cnn_embd, X_VEC)
        X_Concat = tf.concat([word_embd, X_TAG], 1)
        train_inputs.append(X_Concat)

In [14]:
with tf.name_scope("layer_2"):
    outputs = list()
    f_output = tf.Variable(tf.zeros([batch_size,hidden_size],dtype=tf.float32),name='f_output',trainable=False)
    b_output = tf.Variable(tf.zeros([batch_size,hidden_size],dtype=tf.float32),name='b_output',trainable=False)
    for input in (train_inputs):
        f_output = gru_cell(input, f_output, "Forward")
        b_output = gru_cell(tf.reverse(input, [0]), b_output, "Backward")
        outputs.append(tf.layers.dropout(inputs=tf.add(f_output, b_output), rate=dropout_rate, training=training))
    outputs = tf.transpose(outputs, perm=[1, 0, 2])

In [15]:
with tf.name_scope("layer_3"):
    W_arc_dep = tf.tile(w_arc_dep, multiples=[batch_size, 1])
    W_arc_dep = tf.reshape(W_arc_dep, [batch_size, hidden_size, arc_size])
    W_arc_head = tf.tile(w_arc_head, multiples=[batch_size, 1])
    W_arc_head = tf.reshape(W_arc_head, [batch_size, hidden_size, arc_size])
    W_rel_dep = tf.tile(w_rel_dep, multiples=[batch_size, 1])
    W_rel_dep = tf.reshape(W_rel_dep, [batch_size, hidden_size, rel_size])
    W_rel_head = tf.tile(w_rel_head, multiples=[batch_size, 1])
    W_rel_head = tf.reshape(W_rel_head, [batch_size, hidden_size, rel_size])
    B_arc_dep = tf.tile(b_arc_dep, multiples=[batch_size])
    B_arc_dep = tf.reshape(B_arc_dep, [batch_size, arc_size, 1])
    B_arc_head = tf.tile(b_arc_head, multiples=[batch_size])
    B_arc_head = tf.reshape(B_arc_head, [batch_size, arc_size, 1])
    B_rel_dep = tf.tile(b_rel_dep, multiples=[batch_size])
    B_rel_dep = tf.reshape(B_rel_dep, [batch_size, rel_size, 1])
    B_rel_head = tf.tile(b_rel_head, multiples=[batch_size])
    B_rel_head = tf.reshape(B_rel_head, [batch_size, rel_size, 1])
    H_arc_dep = tf.nn.relu(tf.matmul(outputs, W_arc_dep) + b_arc_dep)
    H_arc_dep = tf.layers.dropout(inputs=H_arc_dep, rate=dropout_rate, training=training)
    H_arc_head = tf.nn.relu(tf.matmul(outputs, W_arc_head) + b_arc_head)
    H_arc_head = tf.layers.dropout(inputs=H_arc_head, rate=dropout_rate, training=training)
    H_rel_dep = tf.nn.relu(tf.matmul(outputs, W_rel_dep) + b_rel_dep)
    H_rel_dep = tf.layers.dropout(inputs=H_rel_dep, rate=dropout_rate, training=training)
    H_rel_head = tf.nn.relu(tf.matmul(outputs, W_rel_head) + b_rel_head)
    H_rel_head = tf.layers.dropout(inputs=H_rel_head, rate=dropout_rate, training=training)

In [16]:
W_arc = tf.tile(w_arc, [batch_size, 1])
W_arc = tf.reshape(W_arc, [batch_size, arc_size, arc_size])
B_arc = tf.tile(b_arc, [batch_size, 1])
B_arc = tf.reshape(B_arc, [batch_size, arc_size, 1])

In [17]:
with tf.name_scope("layer_4"):
    arc_score = tf.matmul(tf.matmul(H_arc_head, W_arc), tf.transpose(H_arc_dep, perm=[0,2,1])) + tf.matmul(H_arc_head, B_arc)
    arc_score = tf.layers.dropout(inputs=arc_score, rate=dropout_rate, training=training)
    prediction_head = tf.reshape(tf.argmax(arc_score, 2), [batch_size, sequence_length])

In [18]:
with tf.name_scope("layer_5"):
    rel_score = get_score_rel(H_rel_head, H_rel_dep, prediction_head, w_rel, u_rel, b_rel, sequence_length, "rel_score")
    rel_score = tf.layers.dropout(inputs=rel_score, rate=dropout_rate, training=training)
    rel_score = tf.reshape(rel_score, [batch_size, sequence_length, label_dim])
    prediction_relation = tf.reshape(tf.argmax(rel_score, 2), [batch_size, sequence_length])

In [28]:
save_file = "./model_enable_10000.ckpt"
param_list = [w_forward, u_forward, b_forward, w_backward, u_backward, b_backward,\
              w_arc_dep, w_arc_head, w_rel_dep, w_rel_head,\
              b_arc_head, b_arc_dep, b_rel_dep, b_rel_head,\
              w_arc, b_arc, w_rel, u_rel, b_rel]
batch_generator = mini_batch(batches_morphene, batches_arc, batches_relation)
saver = tf.train.Saver(param_list)
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    saver.restore(sess, save_file)
    X, Y_arc, Y_rel = batch_generator.next_batch(batch_size)
    X_vec, X_char, X_tag = split_data(X)
    feed_dict = dict()
    feed_dict[training] = True
    get_arc = list()
    get_rel = list()
    for ui,(x_vec, x_char, x_tag, y_arc, y_rel) in enumerate(zip(X_vec, X_char, X_tag, Y_arc, Y_rel)):
        feed_dict[train_inputs_vec[ui]]=x_vec
        feed_dict[train_inputs_char[ui]]=x_char
        feed_dict[train_inputs_tag[ui]]=x_tag
        get_arc.append(y_arc)
        get_rel.append(y_rel)
    get_arc = np.transpose(np.asarray(get_arc), axes=[1, 0, 2])
    get_rel = np.transpose(np.asarray(get_rel), axes=[1, 0, 2])
    feed_dict[train_labels_arc] = get_arc
    feed_dict[train_labels_rel] = get_rel
    score_arc, score_rel = sess.run([arc_score, rel_score], feed_dict=feed_dict)

INFO:tensorflow:Restoring parameters from ./model_enable_10000.ckpt


In [29]:
test_tree = enjoy_tree.enjoy_tree(X[0], score_arc[0], score_rel[0])

In [30]:
test_tree.insert_node(False)

In [31]:
test_tree.connect_all_node()

In [32]:
print(test_tree._graph.keys())

dict_keys(['그/MM', '다음/NNG', '에/JKB', '는/JX', '정말/NNG', '이/VCP', '지/EC', '한/MM', '자/NNB', '도/JX', '더/MAG', '쓰/VV', 'ᆯ/ETM', '말/NNG', '이/JKS', '없/VA', '었/EP', '다/EF', './SF'])


In [33]:
tree = test_tree.get_mst()

In [34]:
sentence_index = {}
for i in range(len(test_tree._nodenames)):
    sentence_index[test_tree._nodenames[i]] = i

In [35]:
score_rel = np.argmax(score_rel[0], axis=1)

In [36]:
for k in tree.keys():
    for d in tree[k].keys():
        print("\'{0}\'-> \'{1}\', 관계는 {2}".format(k, d, dictionary_rel[score_rel[sentence_index[d]]]))

print("root -> {0}".format(test_tree._nodenames[test_tree._root]))

'정말/NNG'-> '다음/NNG', 관계는 NNG
'정말/NNG'-> '이/VCP', 관계는 VP
'다음/NNG'-> '에/JKB', 관계는 VP
'이/VCP'-> '는/JX', 관계는 NNG
'이/VCP'-> '정말/NNG', 관계는 VP
'이/VCP'-> '지/EC', 관계는 EF
'이/VCP'-> '한/MM', 관계는 EF
'이/VCP'-> '도/JX', 관계는 VP
'이/VCP'-> '더/MAG', 관계는 NNG
'이/VCP'-> '그/MM', 관계는 VP
'그/MM'-> '자/NNB', 관계는 EF
'그/MM'-> '쓰/VV', 관계는 EP
'그/MM'-> '말/NNG', 관계는 EP
'는/JX'-> 'ᆯ/ETM', 관계는 EP
'지/EC'-> '이/JKS', 관계는 EP
'에/JKB'-> '었/EP', 관계는 EP
'에/JKB'-> '다/EF', 관계는 EP
'다/EF'-> './SF', 관계는 UNK
root -> 없/VA
